In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
sample_id = 'HG01777'

online_candidate_res_root_dir=r'/zfssz2/ST_MCHRI/BIGDATA/PROJECT/NIPT_CNV/f_cnv_out/online/1KGP/'+sample_id+'/cnv_call'
online_re_fname = 'Analysis{}_1000_200_out_cnv.csv'.format(sample_id)
online_fn = os.path.join(online_candidate_res_root_dir, online_re_fname)

in_out_pred = pd.read_csv(online_fn, sep='\t')
in_out_pred.head()

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,CHROM,TRUE_L,CENTOR_INTER_LEN,SUPDUP_INTER_LEN,CENT_CALLLEN_RATIO,SUPDUP_CALLLEN_RATIO
0,51800,52000,200,0.162065,0.237968,0.599967,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
1,73200,75200,2000,0.295456,0.595882,0.108662,1,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
2,358568,358768,200,0.173507,0.011322,0.815171,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
3,361568,361768,200,0.112333,0.099607,0.788060,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
4,362168,363368,1200,0.360907,0.150794,0.488299,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0


In [5]:
gtruth_cnv_base_dir = '/zfssz6/ST_MCHRI/BIGDATA/P18Z10200N0124/NIPT_CNV/NIPT_CNV/1k_cnvs_lab_feat'
gtruth_fn = os.path.join(gtruth_cnv_base_dir, 'ALL.wgs.mergedSV.v8.20130502.svs.genotypes.GRCh38.vcf.' +
        sample_id + '.cnvs.labels')
gtruth_cnv_pd = pd.read_csv(gtruth_fn, sep='\t')
# gtruth_cnv_pd = gtruth_cnv_pd[gtruth_cnv_pd['CHROM']==chr_id]
print('the total number of cnvs for sample {} is {}'.format(sample_id, gtruth_cnv_pd.shape[0]))

true_del_df = gtruth_cnv_pd[gtruth_cnv_pd['ADJTYPE']=='DEL']
true_del_len_arr = true_del_df['SVLEN'].values
print('>'* 4 +'where DEL is {}, length range {}-{}, median: {}, 75% percentil:{}'.format(
    true_del_df.shape[0], min(true_del_len_arr),
    max(true_del_len_arr), np.median(true_del_len_arr), np.percentile(true_del_len_arr,75)))
true_dup_df = gtruth_cnv_pd[gtruth_cnv_pd['ADJTYPE']=='DUP']
true_dup_len_arr = true_dup_df['SVLEN'].values
print('>'* 4 +'where DUP is {}, length range {}-{}, median: {}, 75% percentil:{}'.format(
    true_dup_df.shape[0], min(true_dup_len_arr), 
    max(true_dup_len_arr), np.median(true_dup_len_arr), np.percentile(true_dup_len_arr,75)))

# check truth cnv labels with unpredective region
true_unpredictive_df = in_out_pred[(in_out_pred['PRED_L']==-1) & (~in_out_pred['ID'].isnull())]
uppred_true_lab_lst = np.unique(true_unpredictive_df['ID'].values)
# check all truth cnv labels
true_cnv_df = in_out_pred[(in_out_pred['PRED_L']!=-1) &(~in_out_pred['ID'].isnull())]
pred_true_lab_lst = np.unique(true_cnv_df['ID'].values)

unpred_cnvs = set(uppred_true_lab_lst) - set(pred_true_lab_lst)
unpred_dict = dict()
unpred_dict['DEL']=[]
unpred_dict['DUP']=[]
for i in unpred_cnvs:
    i_lbl = gtruth_cnv_pd.loc[gtruth_cnv_pd['ID']==i, 'ADJTYPE'].values[0]
    unpred_dict[i_lbl].append(i)
print('CNVs that are located at non-read region: ',unpred_dict)

the total number of cnvs for sample HG01777 is 1637
>>>>where DEL is 1517, length range 50-270689, median: 696.0, 75% percentil:2513.0
>>>>where DUP is 120, length range 3070-416253, median: 27713.5, 75% percentil:45470.75
CNVs that are located at non-read region:  {'DEL': [], 'DUP': []}


In [34]:
out_pred=in_out_pred[in_out_pred['LEN']>=200]
# out_pred = in_out_pred

In [35]:
out_pred.shape

(119778, 25)

In [36]:
out_pred.loc[6156]

POS_S                                            151378987
POS_E                                            151381587
LEN                                                   2600
P_NEU                                             0.609497
P_DEL                                             0.386497
P_DUP                                           0.00400603
PRED_L                                                   0
ID                      DUP_uwash_chr1_151333216_151413627
POS                                            1.51361e+08
END                                            1.51441e+08
SVLEN                                                80411
ALT                                            <CN0>,<CN2>
SMP_GT                                         HG01777,1|0
ADJTYPE                                                DEL
ADJAF                                            0.0433307
INTERVAL_LOWER                                   151378987
INTERVAL_UPPER                                   1513815

In [40]:
x = out_pred[(out_pred['TRUE_L']==0)&(out_pred['PRED_L']!=0)]

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,CHROM,TRUE_L,CENTOR_INTER_LEN,SUPDUP_INTER_LEN,CENT_CALLLEN_RATIO,SUPDUP_CALLLEN_RATIO
0,51800,52000,200,0.162065,0.237968,5.999669e-01,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
1,73200,75200,2000,0.295456,0.595882,1.086622e-01,1,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
2,358568,358768,200,0.173507,0.011322,8.151712e-01,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
3,361568,361768,200,0.112333,0.099607,7.880597e-01,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
4,362168,363368,1200,0.360907,0.150794,4.882989e-01,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
5,395168,395368,200,0.006748,0.051759,9.414930e-01,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
6,409768,410768,1000,0.563042,0.010819,4.261388e-01,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
7,422768,423368,600,0.536734,0.010088,4.531785e-01,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
8,438568,438768,200,0.332079,0.017861,6.500596e-01,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0
9,440768,440968,200,0.261728,0.043512,6.947593e-01,2,NaN,NaN,NaN,...,-1,-1,-1,-1,chr1,0,0,0,0.0,0.0


In [37]:
x = out_pred[(out_pred['TRUE_L']==1)&(out_pred['PRED_L']!=1)]

In [39]:
x[x['P_DEL']>=x['P_NEU']]

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,CHROM,TRUE_L,CENTOR_INTER_LEN,SUPDUP_INTER_LEN,CENT_CALLLEN_RATIO,SUPDUP_CALLLEN_RATIO
4538,109689435,109689835,400,0.362015,0.615847,0.022138,0,YL_CN_MSL_117,109682397.0,109702658.0,...,109689435,109689835,400,20261,chr1,1,0,0,0.0,0.0
4539,109689435,109689835,400,0.362015,0.615847,0.022138,0,DUP_gs_CNV_1_110230075_110241247,109687453.0,109698625.0,...,109689435,109689835,400,11172,chr1,1,0,0,0.0,0.0
6170,151419587,151420387,800,0.452834,0.497399,0.049767,0,DUP_uwash_chr1_151333216_151413627,151360740.0,151441151.0,...,151419587,151420387,800,80411,chr1,1,0,0,0.0,0.0
6172,151422787,151425187,2400,0.387488,0.609607,0.002905,0,DUP_uwash_chr1_151333216_151413627,151360740.0,151441151.0,...,151422787,151425187,2400,80411,chr1,1,0,0,0.0,0.0
6646,161605987,161606387,400,0.302191,0.478289,0.219520,0,DUP_gs_CNV_1_161570323_161598180,161600533.0,161628390.0,...,161605987,161606387,400,27857,chr1,1,0,0,0.0,0.0
6653,161626987,161627587,600,0.146905,0.171112,0.681983,2,DUP_gs_CNV_1_161570323_161598180,161600533.0,161628390.0,...,161626987,161627587,600,27857,chr1,1,0,0,0.0,0.0
8162,194483987,194484387,400,0.399578,0.493210,0.107212,0,UW_VH_1353,194481926.0,194485182.0,...,194483987,194484387,400,3256,chr1,1,0,0,0.0,0.0
11931,40219719,40220119,400,0.482715,0.503983,0.013303,0,DEL_pindel_3235,40219806.0,40219933.0,...,40219806,40219933,127,400,chr2,1,0,0,0.0,0.0
15288,113395618,113397218,1600,0.428864,0.435495,0.135641,0,SI_BD_2100,113393736.0,113396037.0,...,113395618,113396037,419,3482,chr2,1,0,0,0.0,0.0
15422,116218618,116221018,2400,0.489570,0.503930,0.006500,0,UW_VH_199,116217061.0,116220721.0,...,116218618,116220721,2103,3957,chr2,1,0,0,0.0,0.0


In [7]:
out_pred.loc[(out_pred['P_DUP'] < 0.8 ) &
                    (out_pred['PRED_L'] == 2), 'PRED_L'] = 0
out_pred.loc[(out_pred['P_DEL'] < 0.8) &
                    (out_pred['PRED_L'] == 1), 'PRED_L'] = 0

/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
out_pred.loc[(np.abs(out_pred['P_DUP'] - out_pred['P_NEU']) < 0.1) &
                    (out_pred['PRED_L'] == 2), 'PRED_L'] = 0
out_pred.loc[(np.abs(out_pred['P_DEL'] - out_pred['P_NEU']) < 0.1) &
                    (out_pred['PRED_L'] == 1), 'PRED_L'] = 0
    # 2 reduce fp
tmp_df = out_pred[['P_NEU', 'P_DEL', 'P_DUP']]
tmp_df.columns = ['0', '1', '2']
out_pred['PRED_P_L'] = tmp_df.idxmax(axis=1, skipna=True)
out_pred['PRED_P_L'] = out_pred['PRED_P_L'].astype(int)
out_pred.loc[(out_pred['PRED_L'] != 0) & (out_pred['PRED_P_L'] == 0), 'PRED_L'] = 0

/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [9]:
true_cnvs = out_pred.loc[out_pred['TRUE_L']!=0,
                         ['ID', 'TRUE_L']].drop_duplicates( 
                     keep = 'first', inplace = False) 

In [10]:
true_dup=true_cnvs[true_cnvs['TRUE_L']==2]
true_del=true_cnvs[true_cnvs['TRUE_L']==1]

In [11]:
 true_del.shape, true_dup.shape

((739, 2), (114, 2))

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,CHROM,TRUE_L,CENTOR_INTER_LEN,SUPDUP_INTER_LEN,CENT_CALLLEN_RATIO,SUPDUP_CALLLEN_RATIO,PRED_P_L
558,16716984,16724984,8000,0.133668,0.003641,0.862691,2,DUP_gs_CNV_1_17026064_17052112,16699569.0,16725617.0,...,16724984,8000,26048,chr1,2,0,0,0.0,0.0,2
563,16884563,16886363,1800,0.154552,0.011802,0.833646,2,DUP_gs_CNV_1_17208619_17225378,16882124.0,16898883.0,...,16886363,1800,16759,chr1,2,0,0,0.0,0.0,2
566,16888763,16890763,2000,0.052100,0.116413,0.831488,2,DUP_gs_CNV_1_17208619_17225378,16882124.0,16898883.0,...,16890763,2000,16759,chr1,2,0,0,0.0,0.0,2
3380,85537635,85539635,2000,0.128632,0.048569,0.822800,2,DUP_gs_CNV_1_85980461_86006144,85514778.0,85540461.0,...,85539635,2000,25683,chr1,2,0,0,0.0,0.0,2
5052,120302635,120303635,1000,0.186876,0.012881,0.800243,2,DUP_gs_CNV_1_144691202_144710622,120286666.0,120306110.0,...,120303635,1000,19444,chr1,2,0,0,0.0,0.0,2
5329,143757187,143758787,1600,0.148356,0.005302,0.846342,2,DUP_gs_CNV_1_149249429_149284877,143754780.0,143790242.0,...,143758787,1600,35462,chr1,2,0,0,0.0,0.0,2
5340,143782187,143784987,2800,0.136776,0.010434,0.852790,2,DUP_gs_CNV_1_149249429_149284877,143754780.0,143790242.0,...,143784987,2800,35462,chr1,2,0,0,0.0,0.0,2
5479,144962187,144963987,1800,0.181054,0.005679,0.813267,2,DUP_gs_CNV_1_143984622_144094733,144915702.0,145023143.0,...,144963987,1800,107441,chr1,2,0,0,0.0,0.0,2
5500,145017987,145018987,1000,0.059062,0.003782,0.937156,2,DUP_gs_CNV_1_143984622_144094733,144915702.0,145023143.0,...,145018987,1000,107441,chr1,2,0,0,0.0,0.0,2
5835,148163587,148164587,1000,0.106287,0.002702,0.891011,2,DUP_uwash_chr1_146081807_146164363,148151364.0,148233918.0,...,148164587,1000,82554,chr1,2,0,0,0.0,0.0,2


In [12]:
false_dup_df = out_pred[(out_pred['PRED_L']==2) & (out_pred['ID'].isnull())]
false_del_df = out_pred[(out_pred['PRED_L']==1) & (out_pred['ID'].isnull())]
false_neg_del_df = out_pred[(out_pred['PRED_L']==0) & (out_pred['TRUE_L']==1)]
false_neg_dup_df = out_pred[(out_pred['PRED_L']==0) & (out_pred['TRUE_L']==2)]
tp_del_df = out_pred[(out_pred['PRED_L']==1) & (out_pred['TRUE_L']==1)]
tp_dup_df = out_pred[(out_pred['PRED_L']==2) & (out_pred['TRUE_L']==2)]

In [13]:
false_dup_df.shape, false_del_df.shape, false_neg_del_df.shape, false_neg_dup_df.shape

((325, 26), (1542, 26), (1364, 26), (1351, 26))

In [14]:
tp_del_df.shape, tp_dup_df.shape

((148, 26), (78, 26))

In [83]:
90/650

0.13846153846153847

In [64]:
false_del_df.head(30)

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,CHROM,TRUE_L,CENTOR_INTER_LEN,SUPDUP_INTER_LEN,CENT_CALLLEN_RATIO,SUPDUP_CALLLEN_RATIO,PRED_P_L
184,6304290,6305490,1200,0.003235,0.996545,0.000220,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
541,15454984,15455984,1000,0.040868,0.955109,0.004023,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
810,22001363,22002363,1000,0.009695,0.969355,0.020950,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
1505,39233835,39234835,1000,0.007488,0.969156,0.023356,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
2178,52850035,52851035,1000,0.027953,0.972020,0.000027,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
2923,70368035,70369035,1000,0.028322,0.971642,0.000036,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
5073,112810635,112811635,1000,0.032546,0.957627,0.009827,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
5616,121585635,121586635,1000,0.042876,0.957091,0.000033,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
5711,143631787,143632987,1200,0.039947,0.958318,0.001735,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
7603,170598787,170599787,1000,0.027567,0.971443,0.000991,1,NaN,NaN,NaN,...,-1,-1,-1,chr1,0,0,0,0.0,0.0,1
